In [4]:
# Dependencies required for recommenders
import numpy as np
import scipy
from lightfm.datasets import fetch_movielens
from lightfm import LightFM

In [11]:
# Fetch the dataset from Movielens with a minimum rating of 5
movies = fetch_movielens(min_rating = 5.0)
#Here, the dataset is stored as a dictionary 'movies' with various string tags to find info.

In [12]:
# See how the data is divided as training and testing data
print(repr(movies['train']))
print(repr(movies['test']))
# notice that testing is almost 1/10th of the training data. This is proper ratio

<943x1682 sparse matrix of type '<type 'numpy.int32'>'
	with 19048 stored elements in COOrdinate format>
<943x1682 sparse matrix of type '<type 'numpy.int32'>'
	with 2153 stored elements in COOrdinate format>


In [15]:
# Building the LightFM model
model = LightFM(loss='warp')
model.fit(movies['train'],epochs = 30,num_threads=2)
# Here, epochs is the times the model goes through the data and the model fits on the training data

In [27]:
# Now that the model is ready, we define a function which provides sample recommendation on the basis of our model
def sample_recommendation(model,data,user_ids):
    # get the items and the user_ids from the dataset
    n_users,n_items = data['train'].shape
    #now, generate a recommendation for each user
    for user_id in user_ids:
        # Here, the data is inmatrix so convert them to a compressed row format
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]
        # predict data by giving the particular user_id and hiis choices
        scores = model.predict(user_id,np.arange(n_items))
        # arrange in descending order to see his top choices
        top_items = data['item_labels'][np.argsort(-scores)]
        
        print("User %s" % user_id)
        print( "    Known Positives:")
        
        for x in known_positives[:3]:
            print( "      %s" % x)
        
        print("   Recommended:")
        for x in top_items[:3]:
            print("     %s" % x)
        


In [29]:
sample_recommendation(model,movies,[12,34,100,1])

User 12
    Known Positives:
      Get Shorty (1995)
      Usual Suspects, The (1995)
      Mighty Aphrodite (1995)
   Recommended:
     Casablanca (1942)
     Pulp Fiction (1994)
     Rear Window (1954)
User 34
    Known Positives:
      Air Force One (1997)
   Recommended:
     Air Force One (1997)
     Titanic (1997)
     Star Wars (1977)
User 100
    Known Positives:
      Jerry Maguire (1996)
   Recommended:
     Titanic (1997)
     Jerry Maguire (1996)
     Good Will Hunting (1997)
User 1
    Known Positives:
      Fargo (1996)
      Godfather, The (1972)
      Kolya (1996)
   Recommended:
     English Patient, The (1996)
     Leaving Las Vegas (1995)
     Good Will Hunting (1997)
